In [1]:
# =============================================================================
# LENDING CLUB LOAN ANALYSIS - PART 1: EDA & PREPROCESSING
# =============================================================================
# This notebook performs exploratory data analysis and preprocessing on the
# LendingClub dataset for loan default prediction
# =============================================================================

# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("=" * 80)
print("LENDING CLUB LOAN DEFAULT PREDICTION PROJECT")
print("=" * 80)
print("\n✓ Libraries imported successfully!\n")

LENDING CLUB LOAN DEFAULT PREDICTION PROJECT

✓ Libraries imported successfully!



In [2]:
# =============================================================================
# STEP 1: LOAD THE DATASET
# =============================================================================
print("STEP 1: Loading Dataset...")
print("-" * 80)

# Load the dataset (adjust path as needed)
# For large files, we'll use chunksize or sample first
try:
    # Try loading full dataset
    df = pd.read_csv('accepted_2007_to_2018Q4.csv', low_memory=False)
    print(f"✓ Dataset loaded successfully!")
    print(f"  Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
except:
    # If file not found, provide instructions
    print("⚠ Dataset file not found. Please ensure 'accepted_2007_to_2018Q4.csv' is in the same directory.")
    print("  You can download it from: https://www.kaggle.com/datasets/wordsforthewise/lending-club")
    # Create sample data structure for demonstration
    df = pd.DataFrame()

STEP 1: Loading Dataset...
--------------------------------------------------------------------------------
✓ Dataset loaded successfully!
  Shape: 2,260,701 rows × 151 columns


In [3]:
# STEP 2: INITIAL DATA EXPLORATION
# =============================================================================
print("\n" + "=" * 80)
print("STEP 2: Initial Data Exploration")
print("=" * 80 + "\n")

# Display basic info
print("2.1 Dataset Overview:")
print("-" * 80)
print(f"Number of Records: {df.shape[0]:,}")
print(f"Number of Features: {df.shape[1]}")
print(f"Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display first few rows
print("\n2.2 First 5 Rows:")
print("-" * 80)
print(df.head())

# Data types
print("\n2.3 Data Types Distribution:")
print("-" * 80)
print(df.dtypes.value_counts())

# Missing values analysis
print("\n2.4 Missing Values Analysis:")
print("-" * 80)
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)
print(f"Columns with missing values: {len(missing_data)}/{len(df.columns)}")
print("\nTop 20 columns with most missing values:")
print(missing_data.head(20))

# Target variable analysis
print("\n2.5 Target Variable (loan_status) Analysis:")
print("-" * 80)
if 'loan_status' in df.columns:
    print(df['loan_status'].value_counts())
    print(f"\nUnique loan statuses: {df['loan_status'].nunique()}")
else:
    print("⚠ loan_status column not found!")


STEP 2: Initial Data Exploration

2.1 Dataset Overview:
--------------------------------------------------------------------------------
Number of Records: 2,260,701
Number of Features: 151
Memory Usage: 6414.67 MB

2.2 First 5 Rows:
--------------------------------------------------------------------------------
         id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  68407277        NaN     3600.0       3600.0           3600.0   36 months   
1  68355089        NaN    24700.0      24700.0          24700.0   36 months   
2  68341763        NaN    20000.0      20000.0          20000.0   60 months   
3  66310712        NaN    35000.0      35000.0          35000.0   60 months   
4  68476807        NaN    10400.0      10400.0          10400.0   60 months   

   int_rate  installment grade sub_grade                    emp_title  \
0     13.99       123.03     C        C4                      leadman   
1     11.99       820.28     C        C1                     En

In [4]:
# =============================================================================
# STEP 3: FEATURE ENGINEERING & SELECTION
# =============================================================================
print("\n" + "=" * 80)
print("STEP 3: Feature Engineering & Selection")
print("=" * 80 + "\n")

# Define important features for loan default prediction
# These features are available BEFORE loan approval (no data leakage)
selected_features = [
    # Loan characteristics
    'loan_amnt',           # Loan amount requested
    'term',                # Loan term (36 or 60 months)
    'int_rate',            # Interest rate
    'installment',         # Monthly payment amount
    'grade',               # LC assigned loan grade
    'sub_grade',           # LC assigned loan subgrade
    
    # Borrower information
    'emp_length',          # Employment length
    'home_ownership',      # Home ownership status
    'annual_inc',          # Annual income
    'verification_status', # Income verification status
    
    # Credit history
    'dti',                 # Debt-to-income ratio
    'delinq_2yrs',        # Number of 30+ days delinquencies in past 2 years
    'inq_last_6mths',     # Number of credit inquiries in last 6 months
    'open_acc',            # Number of open credit lines
    'pub_rec',             # Number of derogatory public records
    'revol_bal',           # Total credit revolving balance
    'revol_util',          # Revolving line utilization rate
    'total_acc',           # Total number of credit lines
    
    # Loan purpose and location
    'purpose',             # Loan purpose
    'addr_state',          # Borrower's state
    
    # Target variable
    'loan_status'          # Current status of the loan
]

print("3.1 Selected Features (No Data Leakage):")
print("-" * 80)
print(f"Total features selected: {len(selected_features)}")
print("\nFeature categories:")
print("  • Loan characteristics: loan_amnt, term, int_rate, installment, grade, sub_grade")
print("  • Borrower info: emp_length, home_ownership, annual_inc, verification_status")
print("  • Credit history: dti, delinq_2yrs, inq_last_6mths, open_acc, pub_rec, revol_bal, revol_util, total_acc")
print("  • Other: purpose, addr_state")

# Filter dataset to selected features
available_features = [col for col in selected_features if col in df.columns]
print(f"\n✓ Available features: {len(available_features)}/{len(selected_features)}")

if len(available_features) > 0:
    df_filtered = df[available_features].copy()
    print(f"✓ Filtered dataset shape: {df_filtered.shape}")
else:
    print("⚠ No selected features found in dataset!")
    df_filtered = df.copy()



STEP 3: Feature Engineering & Selection

3.1 Selected Features (No Data Leakage):
--------------------------------------------------------------------------------
Total features selected: 21

Feature categories:
  • Loan characteristics: loan_amnt, term, int_rate, installment, grade, sub_grade
  • Borrower info: emp_length, home_ownership, annual_inc, verification_status
  • Credit history: dti, delinq_2yrs, inq_last_6mths, open_acc, pub_rec, revol_bal, revol_util, total_acc
  • Other: purpose, addr_state

✓ Available features: 21/21
✓ Filtered dataset shape: (2260701, 21)


In [6]:
# =============================================================================
# STEP 4: DATA CLEANING & PREPROCESSING
# =============================================================================
print("\n" + "=" * 80)
print("STEP 4: Data Cleaning & Preprocessing")
print("=" * 80 + "\n")

# 4.1: Create Binary Target Variable
print("4.1 Creating Binary Target Variable:")
print("-" * 80)

if 'loan_status' in df_filtered.columns:
    # Display original loan statuses
    print("Original loan_status distribution:")
    print(df_filtered['loan_status'].value_counts())
    
    # Define good vs bad loans
    # Good loans: 'Fully Paid', 'Current' (still paying)
    # Bad loans: 'Charged Off', 'Default', 'Late' categories
    good_status = ['Fully Paid', 'Current']
    bad_status = ['Charged Off', 'Default', 'Does not meet the credit policy. Status:Charged Off',
                  'Late (31-120 days)', 'Late (16-30 days)', 'In Grace Period']
    
    # Filter to only completed loans (Fully Paid or Charged Off)
    # Remove current/ongoing loans for better prediction
    completed_loans = ['Fully Paid', 'Charged Off', 'Default', 
                      'Does not meet the credit policy. Status:Charged Off',
                      'Does not meet the credit policy. Status:Fully Paid']
    
    df_filtered = df_filtered[df_filtered['loan_status'].isin(completed_loans)].copy()
    
    # Create binary target: 0 = Fully Paid (Good), 1 = Defaulted (Bad)
    df_filtered['loan_status_binary'] = df_filtered['loan_status'].apply(
        lambda x: 0 if 'Fully Paid' in x else 1
    )
    
    print(f"\n✓ Filtered to completed loans: {len(df_filtered):,} records")
    print("\nBinary target distribution:")
    print(df_filtered['loan_status_binary'].value_counts())
    print(f"\nDefault rate: {df_filtered['loan_status_binary'].mean()*100:.2f}%")
else:
    print("⚠ loan_status column not found!")

# 4.2: Handle Missing Values
print("\n4.2 Handling Missing Values:")
print("-" * 80)

# Check missing values in filtered dataset
missing_filtered = df_filtered.isnull().sum()
missing_filtered = missing_filtered[missing_filtered > 0].sort_values(ascending=False)

print(f"Columns with missing values: {len(missing_filtered)}")
if len(missing_filtered) > 0:
    print("\nMissing value summary:")
    print(missing_filtered)
    
    # Drop columns with >40% missing values
    threshold = 0.4
    cols_to_drop = missing_filtered[missing_filtered > len(df_filtered) * threshold].index.tolist()
    
    if len(cols_to_drop) > 0:
        print(f"\n✓ Dropping {len(cols_to_drop)} columns with >{threshold*100}% missing values:")
        print(cols_to_drop)
        df_filtered = df_filtered.drop(columns=cols_to_drop)
    
    # Drop rows with any remaining missing values (for simplicity)
    initial_rows = len(df_filtered)
    df_filtered = df_filtered.dropna()
    print(f"\n✓ Dropped {initial_rows - len(df_filtered):,} rows with missing values")
    print(f"✓ Remaining records: {len(df_filtered):,}")

# 4.3: Data Type Conversions
print("\n4.3 Data Type Conversions:")
print("-" * 80)

# Convert interest rate from string to float
if 'int_rate' in df_filtered.columns:
    df_filtered['int_rate'] = df_filtered['int_rate'].astype(str).str.replace('%', '').astype(float)
    print("✓ Converted int_rate from percentage string to float")

# Convert revolving utilization from string to float
if 'revol_util' in df_filtered.columns:
    df_filtered['revol_util'] = df_filtered['revol_util'].astype(str).str.replace('%', '').astype(float)
    print("✓ Converted revol_util from percentage string to float")

# Convert term to numeric (extract months)
if 'term' in df_filtered.columns:
    df_filtered['term'] = df_filtered['term'].astype(str).str.extract('(\d+)').astype(float)
    print("✓ Converted term to numeric (months)")

# Convert emp_length to numeric
if 'emp_length' in df_filtered.columns:
    emp_length_map = {
        '< 1 year': 0,
        '1 year': 1,
        '2 years': 2,
        '3 years': 3,
        '4 years': 4,
        '5 years': 5,
        '6 years': 6,
        '7 years': 7,
        '8 years': 8,
        '9 years': 9,
        '10+ years': 10
    }
    df_filtered['emp_length'] = df_filtered['emp_length'].map(emp_length_map)
    print("✓ Converted emp_length to numeric")

print(f"\n✓ Data cleaning completed!")
print(f"  Final dataset shape: {df_filtered.shape}")

# Save cleaned dataset
df_filtered.to_csv('cleaned_lending_data.csv', index=False)
print(f"\n✓ Cleaned dataset saved to 'cleaned_lending_data.csv'")

print("\n" + "=" * 80)
print("PART 1 COMPLETED SUCCESSFULLY!")
print("=" * 80)



STEP 4: Data Cleaning & Preprocessing

4.1 Creating Binary Target Variable:
--------------------------------------------------------------------------------
Original loan_status distribution:
Fully Paid                                             1018715
Charged Off                                             247260
Does not meet the credit policy. Status:Fully Paid        1935
Does not meet the credit policy. Status:Charged Off        740
Default                                                     35
Name: loan_status, dtype: int64

✓ Filtered to completed loans: 1,268,685 records

Binary target distribution:
0    1020650
1     248035
Name: loan_status_binary, dtype: int64

Default rate: 19.55%

4.2 Handling Missing Values:
--------------------------------------------------------------------------------
Columns with missing values: 0

4.3 Data Type Conversions:
--------------------------------------------------------------------------------
✓ Converted int_rate from percentage string